In [1]:
import os
import neuroseries as nts
import astropy.stats as apy

import bk.load
import bk.compute
import bk.plot
import bk.signal
import bk.io

import pandas as pd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.signal
import scipy.stats

In [2]:
def concentration(angles):
    '''
    Compute the kappa parameter Uses the approximation described in "Statistical Analysis of Circular Data" (Fisher, p. 88).
    Translated from MATLAB fmatoolbox.sourceforge.net/Contents/FMAToolbox/General/Concentration.html
    angles : radian

    Copyright (C) 2004-2011 by Michaël Zugaro
    Copyright (C) 2021 by Billel KHOUADER

    '''
    n = len(angles)
    angles = np.exp(1j * angles)  # Complex form of the angles
    r = abs(np.mean(angles))

    if r < 0.53:
        k = 2 * r + r**3 + 5*r**(5/6)
    elif r < 0.85:
        k = -0.4 + 1.39 * r + 0.43 / (1-r)
    else:
        k = 1/(r**3 - 4 * r**2 + 3*r)

    # Correction for very small samples

    if n <= 15:
        if k < 2:
            k = np.max([(k-2)/(n*k), 0])
        else:
            k = (n-1)**3 * k / (n**3+n)

    return k


def mean_resultant_length(angles):
    angles = np.exp(1j*angles)
    return np.abs(np.mean(angles))

In [3]:
def pcc(neuron_phase):

    if isinstance(neuron_phase,nts.Tsd):
        neuron_phase = neuron_phase.values.astype(np.float16)

    pcc = neuron_phase[None,:] - neuron_phase[:,None]
    pcc[np.diag_indices_from(pcc)] = np.nan
    pcc = np.cos(pcc)
    return np.nanmean(pcc,0).mean()


In [4]:
def thetamod(local_path, **args):
    bk.load.current_session_linux(
        base_folder='/mnt/electrophy/Gabrielle/GG-Dataset', local_path=local_path)
    states = bk.load.states()
    print('Loading LFPs')
    lfp = bk.load.lfp_in_intervals(bk.load.ripple_channel(), states['Rem'])
    lfp_filt = bk.signal.passband(lfp, 60, 85)
    power, phase = bk.signal.hilbert(lfp_filt)

    neurons, metadata = bk.load.spikes()
    print(os.getcwd())

    neurons_phase = []
    neurons_rem_fr = []
    for i, n in tqdm(enumerate(neurons)):
        neurons_phase.append(phase.realign(n.restrict(states['Rem'])))
        neurons_rem_fr.append(
            len(n.restrict(states['Rem']))/states['Rem'].tot_length(time_units='s'))

    pvalues = []
    kappas = []
    modulated = []
    MRLs = []
    pccs = []
    for i, p in tqdm(enumerate(neurons_phase)):
        pvalue = apy.rayleightest(p.values)
        pvalues.append(pvalue)

        kappa = concentration(p.values)
        kappas.append(kappa)

        MRL = mean_resultant_length(p.values)
        MRLs.append(MRL)

        phase_consist = pcc(p)
        pccs.append(phase_consist)
        if (pvalue < 0.01) and MRL > 0.04:
            modulated.append(True)
        else:
            modulated.append(False)

    phase_lock = pd.DataFrame({'FR_Rem': neurons_rem_fr,
                               'MRL': MRLs,
                               'Kappa': kappas,
                               'pValue': pvalues,
                               'PPC':pccs,
                               'modulated': modulated})

    # pvalues = pd.DataFrame(pvalues, columns=['pValue'])
    # FRRem = pd.DataFrame(neurons_rem_fr, columns=['FRRem'])

    phase_lock = pd.concat([metadata, phase_lock], axis=1)

    # try:
        # os.makedirs(f'Analysis/Thetamod')
        # os.makedirs(f'/media/billel/DATA/Test/{bk.load.session}')
    # except:
        # print('Already Exist')
    # phase_lock.to_csv(
    #     f'Analysis/Thetamod/{bk.load.session}-theta_mod.csv', index=False)

    # for i in range(len(neurons)):
    #     fig, ax = plt.subplot_mosaic('''
    #                                 A.
    #                                 BC
    #                                 ''',
    #                                  figsize=(12, 8))
    #     fig.set_visible = False
    #     polar_ax = fig.add_subplot(2, 2, 2, polar=True)
    #     ax['A'].plot(lfp.as_units('s'))
    #     ax['A'].plot(lfp_filt.as_units('s'))
    #     plt.sca(ax['A'])
    #     bk.plot.intervals(states['Rem'], time_units='s')
    #     ax['B'].plot(phase.as_units('s'))
    #     ax['B'].plot(neurons_phase[i].as_units('s'), 'ro')

    #     h, a = np.histogram(neurons_phase[i].values, 50)
    #     a = np.convolve(a, [.5, .5], 'same')[1::]

    #     ax['C'].bar(a, h, a[1]-a[0], color='lightblue')
    #     ax['C'].bar(a+2*np.pi, h, a[1]-a[0], color='lightblue')

    # #     ax['C'].bar(neurons_phase[i].values+(2*np.pi),50,'blue')

    #     ax['A'].get_shared_x_axes().join(ax['A'], ax['B'])
    #     polar_ax.bar(a, h, a[1]-a[0])

    #     ax['A'].set_title('LFP')
    #     ax['A'].set_ylim([-500, 500])
    #     ax['A'].set_xlabel('Time (s)')
    #     ax['A'].set_ylabel('Amplitude')

    #     ax['B'].set_title('Phase And Spikes')
    #     ax['B'].set_xlabel('Time (s)')
    #     ax['B'].set_ylabel('Phase (rad)')
    #     polar_ax.set_title('PolarPlot')

    #     ax['C'].set_title('Histogram')
    #     ax['C'].set_xlabel('Phase (rad)')
    #     ax['C'].set_ylabel('Spike Count')

    #     if phase_lock.modulated[i]:
    #         fig.set_facecolor('lightgreen')

    #     plt.suptitle(
    #         f'Neuron #{i},Region = {phase_lock.Region[i]}, Type = {phase_lock.Type[i]}  \nFR_Rem = {phase_lock.FR_Rem[i]} Hz \npValue = {phase_lock.pValue[i]} \n MRL = {phase_lock.MRL[i]}, Kappa = {phase_lock.Kappa[i]} \n Modulated : {phase_lock.modulated[i]}')
    #     plt.tight_layout()

    #     plt.savefig(
    #         f'/media/billel/DATA/Test/{bk.load.session}/Neuron#{i}-{metadata.Shank[i]}-{metadata.Id[i]}.png')
    #     print(f'Saved Analysis/ThetaMod/Neuron#{i}-{metadata.Shank[i]}-{metadata.Id[i]}.png')
    #     fig.clear()
    #     plt.close(fig)

    del lfp, lfp_filt, neurons, neurons_phase
    return phase_lock


In [5]:
phase_lock = thetamod('Rat08/Rat08-20130713')

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /mnt/electrophy/Gabrielle/GG-Dataset/Rat08/Rat08-20130713
Loading LFPs
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
/mnt/electrophy/Gabrielle/GG-Dataset/Rat08/Rat08-20130713


252it [00:03, 69.21it/s]
252it [06:50,  1.63s/it]


In [7]:
def gammamod(local_path, **args):
    bk.load.current_session_linux(
        base_folder='/mnt/electrophy/Gabrielle/GG-Dataset', local_path=local_path)
    states = bk.load.states()
    print('Loading LFPs')
    lfp = bk.load.lfp_in_intervals(bk.load.random_channel('BLA'), states['Rem'])
    lfp_filt = bk.signal.passband(lfp, 60, 85)
    power, phase = bk.signal.hilbert(lfp_filt)

    neurons, metadata = bk.load.spikes()
    print(os.getcwd())

    neurons_phase = []
    neurons_rem_fr = []
    for i, n in tqdm(enumerate(neurons)):
        neurons_phase.append(phase.realign(n.restrict(states['Rem'])))
        neurons_rem_fr.append(
            len(n.restrict(states['Rem']))/states['Rem'].tot_length(time_units='s'))

    pvalues = []
    kappas = []
    modulated = []
    MRLs = []
    pccs = []
    for i, p in tqdm(enumerate(neurons_phase)):
        pvalue = apy.rayleightest(p.values)
        pvalues.append(pvalue)

        kappa = concentration(p.values)
        kappas.append(kappa)

        MRL = mean_resultant_length(p.values)
        MRLs.append(MRL)

        phase_consist = pcc(p)
        pccs.append(phase_consist)
        if (pvalue < 0.01) and MRL > 0.04:
            modulated.append(True)
        else:
            modulated.append(False)

    phase_lock = pd.DataFrame({'FR_Rem': neurons_rem_fr,
                               'MRL': MRLs,
                               'Kappa': kappas,
                               'pValue': pvalues,
                               'PPC':pccs,
                               'modulated': modulated})

    # pvalues = pd.DataFrame(pvalues, columns=['pValue'])
    # FRRem = pd.DataFrame(neurons_rem_fr, columns=['FRRem'])

    phase_lock = pd.concat([metadata, phase_lock], axis=1)

    # try:
        # os.makedirs(f'Analysis/Thetamod')
        # os.makedirs(f'/media/billel/DATA/Test/{bk.load.session}')
    # except:
        # print('Already Exist')
    # phase_lock.to_csv(
    #     f'Analysis/Thetamod/{bk.load.session}-theta_mod.csv', index=False)

    # for i in range(len(neurons)):
    #     fig, ax = plt.subplot_mosaic('''
    #                                 A.
    #                                 BC
    #                                 ''',
    #                                  figsize=(12, 8))
    #     fig.set_visible = False
    #     polar_ax = fig.add_subplot(2, 2, 2, polar=True)
    #     ax['A'].plot(lfp.as_units('s'))
    #     ax['A'].plot(lfp_filt.as_units('s'))
    #     plt.sca(ax['A'])
    #     bk.plot.intervals(states['Rem'], time_units='s')
    #     ax['B'].plot(phase.as_units('s'))
    #     ax['B'].plot(neurons_phase[i].as_units('s'), 'ro')

    #     h, a = np.histogram(neurons_phase[i].values, 50)
    #     a = np.convolve(a, [.5, .5], 'same')[1::]

    #     ax['C'].bar(a, h, a[1]-a[0], color='lightblue')
    #     ax['C'].bar(a+2*np.pi, h, a[1]-a[0], color='lightblue')

    # #     ax['C'].bar(neurons_phase[i].values+(2*np.pi),50,'blue')

    #     ax['A'].get_shared_x_axes().join(ax['A'], ax['B'])
    #     polar_ax.bar(a, h, a[1]-a[0])

    #     ax['A'].set_title('LFP')
    #     ax['A'].set_ylim([-500, 500])
    #     ax['A'].set_xlabel('Time (s)')
    #     ax['A'].set_ylabel('Amplitude')

    #     ax['B'].set_title('Phase And Spikes')
    #     ax['B'].set_xlabel('Time (s)')
    #     ax['B'].set_ylabel('Phase (rad)')
    #     polar_ax.set_title('PolarPlot')

    #     ax['C'].set_title('Histogram')
    #     ax['C'].set_xlabel('Phase (rad)')
    #     ax['C'].set_ylabel('Spike Count')

    #     if phase_lock.modulated[i]:
    #         fig.set_facecolor('lightgreen')

    #     plt.suptitle(
    #         f'Neuron #{i},Region = {phase_lock.Region[i]}, Type = {phase_lock.Type[i]}  \nFR_Rem = {phase_lock.FR_Rem[i]} Hz \npValue = {phase_lock.pValue[i]} \n MRL = {phase_lock.MRL[i]}, Kappa = {phase_lock.Kappa[i]} \n Modulated : {phase_lock.modulated[i]}')
    #     plt.tight_layout()

    #     plt.savefig(
    #         f'/media/billel/DATA/Test/{bk.load.session}/Neuron#{i}-{metadata.Shank[i]}-{metadata.Id[i]}.png')
    #     print(f'Saved Analysis/ThetaMod/Neuron#{i}-{metadata.Shank[i]}-{metadata.Id[i]}.png')
    #     fig.clear()
    #     plt.close(fig)

    del lfp, lfp_filt, neurons, neurons_phase
    return phase_lock


In [8]:
phase_lock = gammamod('Rat08/Rat08-20130713')

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /mnt/electrophy/Gabrielle/GG-Dataset/Rat08/Rat08-20130713
Loading LFPs
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
/mnt/electrophy/Gabrielle/GG-Dataset/Rat08/Rat08-20130713


252it [00:03, 68.75it/s]
84it [01:20,  3.54s/it]

In [ ]:
phase_lock

In [24]:
import seaborn as sns
%matplotlib qt

stru =  ['Hpc','BLA']

fig,ax = plt.subplots(1,2)

for i,s in enumerate(stru):
    plt.sca(ax[i])
    sns.scatterplot(data = phase_lock[phase_lock.Region == s],y = 'MRL',x = 'PPC',hue = 'modulated',palette='viridis')
    plt.xlim(0,0.4)
    plt.ylim(0,0.7)
    plt.title(s)
plt.tight_layout()


In [25]:
np.argmax(phase_lock[phase_lock.Region == 'BLA'].MRL)

81

In [28]:
phase_lock[phase_lock.Region == 'BLA'].iloc[81]

Rat                 8
Day                 6
Shank              18
Id                 15
Region            BLA
Type              Pyr
FR_Rem       0.041333
MRL          0.183889
Kappa        1.593262
pValue       0.036375
PPC          0.023819
modulated       False
Name: 239, dtype: object

In [9]:
len(neurons[84])

61567

In [4]:
%matplotlib Agg
b = bk.load.batch(thetamod,linux = True)

()


  0%|          | 0/61 [00:00<?, ?it/s]

Loading Data from Rat08-20130708
Rat : 8 on day : 1
Working with session Rat08-20130708 @ /mnt/electrophy/Gabrielle/GG-Dataset/Rat08/Rat08-20130708
Loading LFPs


In [5]:
all_thetamod = []
for session,data in b.items():
    if np.any(data): all_thetamod.append(data)

all_thetamod = pd.concat(all_thetamod)
all_thetamod.to_csv(f'{bk.load.base}/All-Rats/Billel/thetamod.csv',index = False)

In [36]:
def load_thetamod(local_path):

    bk.load.current_session_linux(
        base_folder='/mnt/electrophy/Gabrielle/GG-Dataset', local_path=local_path)


    df = bk.io.load_dataframe(f'Thetamod/{bk.load.session}-theta_mod')

    return df


Analysis/Thetamod/Rat10-20140703-theta_mod.csv
Could not find the requested analysis file


SyntaxError: 'return' outside function (2348186710.py, line 9)

### Using PPC

In [1]:
import bk.load
import bk.signal

import matplotlib.pyplot as plt
import numpy as np


In [ ]:
def pcc(neuron_phase):

    if isinstance(neuron_phase,nts.Tsd):
        neuron_phase = neuron_phase.values.astype(np.float32)

    pcc = neuron_phase[None,:] - neuron_phase[:,None]
    pcc[np.diag_indices_from(pcc)] = np.nan
    # pcc = np.cos(pcc.ravel())
    return np.nanmean(pcc)


In [3]:
bk.load.current_session_linux()
neurons,metadata = bk.load.spikes()
states = bk.load.states()

for i,n in enumerate(neurons):
    neurons[i] = n.restrict(states['Rem'])

lfp = bk.load.lfp_in_intervals(23,states['Rem'])
lfp_filt = bk.signal.passband(lfp,4,12)
power,phase = bk.signal.hilbert(lfp_filt)

neurons_phase = []
neurons_rem_fr = []
for i, n in enumerate(neurons):
    neurons_phase.append(phase.realign(n.restrict(states['Rem'])))
    neurons_rem_fr.append(
        len(n.restrict(states['Rem']))/states['Rem'].tot_length(time_units='s'))

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
